# Sesión 18: Práctica Intensiva con Archivos 📊

**Curso:** Estud-IA Programación G57  
**Docente:** Eldigardo Camacho  
**Duración:** ~3 horas

---
## ✅ Checklist de Apertura

Antes de empezar, verifica:
- [ ] Sé abrir y leer archivos con `with open()`
- [ ] Conozco los modos 'r', 'w', 'a'
- [ ] Puedo usar `csv.DictReader()` y `csv.DictWriter()`

---
## 🎯 Objetivos de la Sesión

Esta es una **sesión práctica intensiva**. Al final, habrás:
1. Construido un **sistema de gestión de inventario**
2. Implementado **CRUD** (Crear, Leer, Actualizar, Eliminar)
3. Generado **reportes** a partir de datos CSV
4. Manejado **errores** de archivos
5. Creado **backups** de datos

---
## 🎬 Proyecto: Sistema de Gestión de Inventario

Vamos a construir un sistema completo paso a paso.

In [ ]:
# Preparación: Descargar archivos necesarios
!wget -q https://raw.githubusercontent.com/heldigard/estudia-programacion-g57/master/estudiantes/datasets/ventas.csv
!wget -q https://raw.githubusercontent.com/heldigard/estudia-programacion-g57/master/estudiantes/datasets/contactos.csv
print('✅ Archivos descargados!')

### Paso 1: Crear el inventario inicial

In [ ]:
import csv
from datetime import datetime

# Crear inventario inicial
inventario_inicial = [
    {'codigo': 'P001', 'producto': 'Laptop HP', 'cantidad': 15, 'precio': 2500000, 'categoria': 'Computadores'},
    {'codigo': 'P002', 'producto': 'Mouse Logitech', 'cantidad': 50, 'precio': 45000, 'categoria': 'Accesorios'},
    {'codigo': 'P003', 'producto': 'Teclado Mecánico', 'cantidad': 30, 'precio': 120000, 'categoria': 'Accesorios'},
    {'codigo': 'P004', 'producto': 'Monitor Samsung', 'cantidad': 20, 'precio': 850000, 'categoria': 'Monitores'},
    {'codigo': 'P005', 'producto': 'Audífonos Sony', 'cantidad': 40, 'precio': 180000, 'categoria': 'Audio'},
    {'codigo': 'P006', 'producto': 'Webcam HD', 'cantidad': 25, 'precio': 95000, 'categoria': 'Video'},
    {'codigo': 'P007', 'producto': 'Disco SSD 500GB', 'cantidad': 35, 'precio': 280000, 'categoria': 'Almacenamiento'},
    {'codigo': 'P008', 'producto': 'Impresora Epson', 'cantidad': 10, 'precio': 680000, 'categoria': 'Impresoras'},
]

def crear_inventario(productos, archivo='inventario.csv'):
    """Crea el archivo de inventario inicial."""
    campos = ['codigo', 'producto', 'cantidad', 'precio', 'categoria']
    
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        escritor = csv.DictWriter(f, fieldnames=campos)
        escritor.writeheader()
        escritor.writerows(productos)
    
    print(f'✅ Inventario creado con {len(productos)} productos')

crear_inventario(inventario_inicial)

### Paso 2: Funciones CRUD

In [ ]:
# === LEER (Read) ===

def cargar_inventario(archivo='inventario.csv'):
    """Carga todos los productos del inventario."""
    productos = []
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            lector = csv.DictReader(f)
            for fila in lector:
                # Convertir tipos
                fila['cantidad'] = int(fila['cantidad'])
                fila['precio'] = int(fila['precio'])
                productos.append(fila)
    except FileNotFoundError:
        print(f'❌ Archivo {archivo} no encontrado')
    return productos

def buscar_producto(codigo, productos):
    """Busca un producto por su código."""
    for producto in productos:
        if producto['codigo'] == codigo:
            return producto
    return None

# Probar
inventario = cargar_inventario()
print(f'Productos cargados: {len(inventario)}')
print(f'\nBuscando P003: {buscar_producto("P003", inventario)}')

In [ ]:
# === GUARDAR ===

def guardar_inventario(productos, archivo='inventario.csv'):
    """Guarda la lista de productos en el archivo CSV."""
    campos = ['codigo', 'producto', 'cantidad', 'precio', 'categoria']
    
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        escritor = csv.DictWriter(f, fieldnames=campos)
        escritor.writeheader()
        escritor.writerows(productos)
    
    return True

In [ ]:
# === CREAR (Create) ===

def agregar_producto(nuevo_producto, archivo='inventario.csv'):
    """Agrega un nuevo producto al inventario."""
    productos = cargar_inventario(archivo)
    
    # Verificar si ya existe
    if buscar_producto(nuevo_producto['codigo'], productos):
        print(f'❌ El código {nuevo_producto["codigo"]} ya existe')
        return False
    
    productos.append(nuevo_producto)
    guardar_inventario(productos, archivo)
    print(f'✅ Producto {nuevo_producto["producto"]} agregado')
    return True

# Probar
nuevo = {
    'codigo': 'P009',
    'producto': 'Tablet Samsung',
    'cantidad': 12,
    'precio': 1200000,
    'categoria': 'Tablets'
}
agregar_producto(nuevo)

In [ ]:
# === ACTUALIZAR (Update) ===

def actualizar_cantidad(codigo, nueva_cantidad, archivo='inventario.csv'):
    """Actualiza la cantidad de un producto."""
    productos = cargar_inventario(archivo)
    
    for producto in productos:
        if producto['codigo'] == codigo:
            cantidad_anterior = producto['cantidad']
            producto['cantidad'] = nueva_cantidad
            guardar_inventario(productos, archivo)
            print(f'✅ {producto["producto"]}: {cantidad_anterior} → {nueva_cantidad}')
            return True
    
    print(f'❌ Producto {codigo} no encontrado')
    return False

def actualizar_precio(codigo, nuevo_precio, archivo='inventario.csv'):
    """Actualiza el precio de un producto."""
    productos = cargar_inventario(archivo)
    
    for producto in productos:
        if producto['codigo'] == codigo:
            precio_anterior = producto['precio']
            producto['precio'] = nuevo_precio
            guardar_inventario(productos, archivo)
            print(f'✅ {producto["producto"]}: ${precio_anterior:,} → ${nuevo_precio:,}')
            return True
    
    print(f'❌ Producto {codigo} no encontrado')
    return False

# Probar
actualizar_cantidad('P002', 45)
actualizar_precio('P001', 2400000)

In [ ]:
# === ELIMINAR (Delete) ===

def eliminar_producto(codigo, archivo='inventario.csv'):
    """Elimina un producto del inventario."""
    productos = cargar_inventario(archivo)
    
    for i, producto in enumerate(productos):
        if producto['codigo'] == codigo:
            eliminado = productos.pop(i)
            guardar_inventario(productos, archivo)
            print(f'✅ Producto {eliminado["producto"]} eliminado')
            return True
    
    print(f'❌ Producto {codigo} no encontrado')
    return False

# Probar (cuidado, esto elimina de verdad)
# eliminar_producto('P009')

### Paso 3: Reportes

In [ ]:
def mostrar_inventario(archivo='inventario.csv'):
    """Muestra el inventario en formato tabla."""
    productos = cargar_inventario(archivo)
    
    print('=' * 80)
    print(f'{"CÓDIGO":<8} {"PRODUCTO":<20} {"CANT":>6} {"PRECIO":>12} {"CATEGORIA":<15}')
    print('=' * 80)
    
    for p in productos:
        print(f'{p["codigo"]:<8} {p["producto"]:<20} {p["cantidad"]:>6} ${p["precio"]:>10,} {p["categoria"]:<15}')
    
    print('=' * 80)
    print(f'Total de productos: {len(productos)}')

mostrar_inventario()

In [ ]:
def reporte_valor_inventario(archivo='inventario.csv'):
    """Calcula el valor total del inventario."""
    productos = cargar_inventario(archivo)
    
    print('\n📊 REPORTE DE VALOR DEL INVENTARIO')
    print('=' * 60)
    
    valor_total = 0
    for p in productos:
        valor_producto = p['cantidad'] * p['precio']
        valor_total += valor_producto
        print(f'{p["producto"]:<25} {p["cantidad"]:>4} x ${p["precio"]:>10,} = ${valor_producto:>12,}')
    
    print('=' * 60)
    print(f'{"VALOR TOTAL DEL INVENTARIO:":>45} ${valor_total:>12,}')
    
    return valor_total

reporte_valor_inventario()

In [ ]:
def reporte_por_categoria(archivo='inventario.csv'):
    """Agrupa productos por categoría."""
    productos = cargar_inventario(archivo)
    
    # Agrupar por categoría
    categorias = {}
    for p in productos:
        cat = p['categoria']
        if cat not in categorias:
            categorias[cat] = {'productos': 0, 'unidades': 0, 'valor': 0}
        categorias[cat]['productos'] += 1
        categorias[cat]['unidades'] += p['cantidad']
        categorias[cat]['valor'] += p['cantidad'] * p['precio']
    
    print('\n📊 REPORTE POR CATEGORÍA')
    print('=' * 70)
    print(f'{"CATEGORÍA":<20} {"PRODUCTOS":>10} {"UNIDADES":>10} {"VALOR":>15}')
    print('-' * 70)
    
    for cat, datos in sorted(categorias.items()):
        print(f'{cat:<20} {datos["productos"]:>10} {datos["unidades"]:>10} ${datos["valor"]:>12,}')
    
    print('=' * 70)

reporte_por_categoria()

In [ ]:
def productos_bajo_stock(minimo=20, archivo='inventario.csv'):
    """Lista productos con stock bajo."""
    productos = cargar_inventario(archivo)
    
    bajo_stock = [p for p in productos if p['cantidad'] < minimo]
    
    print(f'\n⚠️ PRODUCTOS CON STOCK BAJO (< {minimo} unidades)')
    print('=' * 50)
    
    if bajo_stock:
        for p in bajo_stock:
            print(f'🔴 {p["codigo"]} - {p["producto"]}: {p["cantidad"]} unidades')
    else:
        print('✅ No hay productos con stock bajo')
    
    return bajo_stock

productos_bajo_stock(15)

### Paso 4: Backup y Registro de Cambios

In [ ]:
import shutil
from datetime import datetime

def crear_backup(archivo='inventario.csv'):
    """Crea una copia de seguridad del inventario."""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_nombre = f'backup_{timestamp}_{archivo}'
    
    try:
        shutil.copy(archivo, backup_nombre)
        print(f'✅ Backup creado: {backup_nombre}')
        return backup_nombre
    except FileNotFoundError:
        print(f'❌ No se encontró {archivo}')
        return None

crear_backup()

In [ ]:
def registrar_movimiento(tipo, producto, cantidad, archivo_log='movimientos.txt'):
    """Registra un movimiento en el log."""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    registro = f'[{timestamp}] {tipo.upper()}: {producto} - Cantidad: {cantidad}\n'
    
    with open(archivo_log, 'a', encoding='utf-8') as f:
        f.write(registro)
    
    print(f'📝 Movimiento registrado')

# Ejemplo de uso
registrar_movimiento('ENTRADA', 'Laptop HP', 5)
registrar_movimiento('SALIDA', 'Mouse Logitech', 3)
registrar_movimiento('AJUSTE', 'Monitor Samsung', -2)

# Ver el log
print('\n📋 LOG DE MOVIMIENTOS:')
with open('movimientos.txt', 'r', encoding='utf-8') as f:
    print(f.read())

### Paso 5: Menú Interactivo

---
## Actividad Extra (10-15 min): Filtrar contactos por edad

Objetivo: practicar DictReader y DictWriter con filtros.

Pasos sugeridos:
1. Lee contactos.csv
2. Filtra personas con edad >= 30
3. Guarda el resultado en un nuevo CSV

In [ ]:
# === MINI-PROYECTO: FILTRO DE CONTACTOS ===
import csv

ruta_in = "../datasets/contactos.csv"
ruta_out = "contactos_filtrados.csv"

with open(ruta_in, newline="", encoding="utf-8") as f, open(ruta_out, "w", newline="", encoding="utf-8") as g:
    reader = csv.DictReader(f)
    writer = csv.DictWriter(g, fieldnames=reader.fieldnames)
    writer.writeheader()
    for row in reader:
        if int(row["edad"]) >= 30:
            writer.writerow(row)

print("Listo:", ruta_out)

In [ ]:
def mostrar_menu():
    """Muestra el menú principal."""
    print('\n' + '=' * 40)
    print('  📦 SISTEMA DE GESTIÓN DE INVENTARIO')
    print('=' * 40)
    print('1. Ver inventario')
    print('2. Buscar producto')
    print('3. Agregar producto')
    print('4. Actualizar cantidad')
    print('5. Actualizar precio')
    print('6. Eliminar producto')
    print('7. Reporte de valor')
    print('8. Reporte por categoría')
    print('9. Productos con bajo stock')
    print('10. Crear backup')
    print('0. Salir')
    print('-' * 40)

def sistema_inventario():
    """Sistema principal con menú interactivo."""
    while True:
        mostrar_menu()
        opcion = input('Seleccione una opción: ')
        
        if opcion == '0':
            print('\n👋 ¡Hasta luego!')
            break
        elif opcion == '1':
            mostrar_inventario()
        elif opcion == '2':
            codigo = input('Código del producto: ').upper()
            productos = cargar_inventario()
            producto = buscar_producto(codigo, productos)
            if producto:
                print(f'\n✅ Encontrado: {producto}')
            else:
                print(f'\n❌ Producto {codigo} no encontrado')
        elif opcion == '3':
            nuevo = {
                'codigo': input('Código: ').upper(),
                'producto': input('Nombre: '),
                'cantidad': int(input('Cantidad: ')),
                'precio': int(input('Precio: ')),
                'categoria': input('Categoría: ')
            }
            agregar_producto(nuevo)
        elif opcion == '4':
            codigo = input('Código del producto: ').upper()
            cantidad = int(input('Nueva cantidad: '))
            actualizar_cantidad(codigo, cantidad)
        elif opcion == '5':
            codigo = input('Código del producto: ').upper()
            precio = int(input('Nuevo precio: '))
            actualizar_precio(codigo, precio)
        elif opcion == '6':
            codigo = input('Código del producto a eliminar: ').upper()
            confirmar = input(f'¿Seguro que desea eliminar {codigo}? (s/n): ')
            if confirmar.lower() == 's':
                eliminar_producto(codigo)
        elif opcion == '7':
            reporte_valor_inventario()
        elif opcion == '8':
            reporte_por_categoria()
        elif opcion == '9':
            minimo = int(input('Stock mínimo a verificar: '))
            productos_bajo_stock(minimo)
        elif opcion == '10':
            crear_backup()
        else:
            print('\n❌ Opción no válida')
        
        input('\nPresione Enter para continuar...')

# Descomentar para ejecutar el sistema interactivo
# sistema_inventario()

---
## 💪 Ejercicios de Práctica

### Ejercicio 1: Exportar a reporte

In [ ]:
# Crea una función que exporte el inventario a un archivo de texto
# formateado como reporte

def exportar_reporte_txt(archivo_salida='reporte_inventario.txt'):
    """Exporta el inventario a un archivo de texto formateado."""
    # Tu código aquí:
    pass

# Prueba tu función
# exportar_reporte_txt()

### Ejercicio 2: Análisis de ventas

In [ ]:
# Usando el archivo ventas.csv, crea funciones para:
# 1. Encontrar el día con más ventas
# 2. El vendedor con más ventas totales
# 3. El producto más vendido (por cantidad)

# Tu código aquí:

### Ejercicio 3: Combinar archivos

In [ ]:
# Crea una función que combine las ventas con la información de contactos
# para generar un reporte de ventas por vendedor con sus datos de contacto

# Tu código aquí:

---
## ⚠️ Manejo de Errores con Archivos

In [ ]:
# Siempre manejar errores al trabajar con archivos

def leer_archivo_seguro(nombre_archivo):
    """Lee un archivo manejando posibles errores."""
    try:
        with open(nombre_archivo, 'r', encoding='utf-8') as f:
            contenido = f.read()
            return contenido
    except FileNotFoundError:
        print(f'❌ Error: El archivo "{nombre_archivo}" no existe')
        return None
    except PermissionError:
        print(f'❌ Error: No tienes permiso para leer "{nombre_archivo}"')
        return None
    except Exception as e:
        print(f'❌ Error inesperado: {e}')
        return None

# Probar con archivo que existe
print('Archivo existente:')
resultado = leer_archivo_seguro('inventario.csv')
if resultado:
    print('✅ Archivo leído correctamente')

# Probar con archivo que NO existe
print('\nArchivo inexistente:')
resultado = leer_archivo_seguro('no_existe.txt')

---
## 📝 Mini-Quiz de Cierre

**1.** ¿Qué significa CRUD?
- a) Create, Read, Update, Delete
- b) Copy, Read, Use, Download
- c) Create, Remove, Update, Drop

**2.** ¿Qué módulo usamos para copiar archivos?
- a) copy
- b) shutil
- c) os

**3.** ¿Para qué sirve `try-except` al trabajar con archivos?
- a) Para leer más rápido
- b) Para manejar errores sin que el programa se detenga
- c) Para escribir archivos

**4.** ¿Cuál es la mejor práctica para registrar cambios?
- a) Solo guardar el archivo
- b) Mantener un log de movimientos
- c) Imprimir en pantalla

**5.** ¿Por qué es importante crear backups?
- a) Para usar menos memoria
- b) Para recuperar datos en caso de errores
- c) Para que el programa sea más rápido

<details>
<summary>👀 Ver respuestas</summary>

1. **a) Create, Read, Update, Delete**
2. **b) shutil** - Para operaciones de archivos de alto nivel
3. **b) Para manejar errores sin que el programa se detenga**
4. **b) Mantener un log de movimientos**
5. **b) Para recuperar datos en caso de errores**

</details>

---
## 🎮 Actividades Interactivas

### 🧠 Actividad: El Analista Junior
**Rol:** Eres un analista de datos junior en una tienda.
**Misión:** Tu jefe te ha dado `ventas.csv` y quiere saber:
1. ¿Cuál es el producto "Estrella" (más ingresos generados)?
2. ¿Qué vendedor merece un bono (más ventas totales)?

¡Usa Python para responderle rápido!

### 🐞 Debugging en Vivo
Errores en lógica de negocios:

In [ ]:
# Error: Convertir precios
# precio = row['precio'] # Es string '15000'
# total = precio * 2 # Resultado '1500015000' (String repetido)

# Error: Contador mal puesto
# for row in reader:
#     total = 0 # ¡Reinicia el total en cada fila!
#     total += int(row['precio'])

### 🚀 Proyecto Colaborativo: Generador de Informes HTML
**Reto Avanzado:**
Lee `ventas.csv` y genera un archivo `reporte.html` simple que tenga:
- Un título `<h1>Reporte de Ventas</h1>`
- Una lista `<ul>` con los productos y sus precios.
- Abre el archivo generado en tu navegador.

In [ ]:
# Tu código aquí:



---

## 🌐 ¡Practica en la Web!

¿Quieres ver cómo funcionan los sistemas CRUD en JavaScript?

**Abre la plantilla web de S18** en tu navegador o en [CodePen.io](https://codepen.io/pen):

```
estudiantes/templates/web_sessions/s18/
```

Encontrarás ejercicios interactivos para practicar:
- Sistema CRUD completo (Crear, Leer, Actualizar, Eliminar)
- Manipulación de datos JSON
- Actualización de DOM en tiempo real
- Persistencia con localStorage

**¡JavaScript permite crear aplicaciones web completas con persistencia!**
- `csv.DictWriter` → `JSON.stringify()` y `localStorage`
- `with open()` → `fetch()` para recursos
- Operaciones CRUD son similares en lógica


---
## Sección Web (S18) - Tablas y Filtrado

### 🌐 Tablas de Datos: Python vs JavaScript

**IMPORTANTE:** En la web, las tablas HTML (no archivos CSV) son la forma
más común de mostrar datos. JavaScript permite manipular estas tablas
en tiempo real con búsqueda, filtrado y ordenamiento.

---

### 📖 Comparación de Conceptos

| Operación | Python (pandas/CSV) | JavaScript (HTML/DOM) |
|------------|---------------------|----------------------|
| Leer datos | `csv.DictReader()` | `fetch()` + `response.json()` |
| Filtrar | `[x for x in data if x.campo > valor]` | `data.filter(x => x.campo > valor)` |
| Ordenar | `sorted(data, key=lambda x: x.campo)` | `data.sort((a,b) => a.campo - b.campo)` |
| Agrupar | `itertools.groupby()` | `data.reduce()` |
| Paginar | `datos[inicio:fin]` | `datos.slice(inicio, fin)` |
| Mostrar | `print()` | `tbody.innerHTML` |

---

### 📊 Estructura de Tabla HTML

Las tablas HTML se estructuran así:

```html
<table>
  <thead>                    <!-- Cabecera (títulos) -->
    <tr>                    <!-- Fila de encabezados -->
      <th>Nombre</th>       <!-- Celda de título -->
      <th>Email</th>
    </tr>
  </thead>
  <tbody>                    <!-- Cuerpo (datos) -->
    <tr>                    <!-- Fila de datos -->
      <td>Ana</td>          <!-- Celda de dato -->
      <td>ana@email.com</td>
    </tr>
  </tbody>
</table>
```

**Equivalente Python (print con formato):**
```python
print(f"{'Nombre':<20} {'Email':<30}")
print("-" * 50)
print(f"{'Ana':<20} {'ana@email.com':<30}")
```

---

### 🔍 Filtrado en Tiempo Real

**Python:** Filtras una vez y muestras el resultado
```python
# Filtrar estudiantes con nota > 80
filtrados = [e for e in estudiantes if e['nota'] > 80]
for est in filtrados:
    print(est)
```

**JavaScript:** Filtras mientras el usuario escribe
```javascript
// Filtrar mientras escribe (búsqueda en tiempo real)
busquedaInput.addEventListener('input', (e) => {
  const texto = e.target.value.toLowerCase();
  const filtrados = estudiantes.filter(est => 
    est.nombre.toLowerCase().includes(texto)
  );
  renderTabla(filtrados);  // Actualizar tabla HTML
});
```

---

### ⬆️⬇️ Ordenamiento de Columnas

**Python (pandas):**
```python
import pandas as pd

df = pd.DataFrame(estudiantes)
df_ordenado = df.sort_values('nota', ascending=False)
print(df_ordenado)
```

**JavaScript:**
```javascript
// Ordenar por nota ( descendente)
const ordenados = [...estudiantes].sort((a, b) => b.nota - a.nota);
renderTabla(ordenados);
```

---

### 📄 Paginación

**Python:**
```python
# Mostrar página 2 (elementos 6-10)
pagina = estudiantes[5:10]
for est in pagina:
    print(est)
```

**JavaScript:**
```javascript
// Mostrar página 2 (elementos 6-10)
const porPagina = 5;
const pagina = 2;
const inicio = (pagina - 1) * porPagina;
const fin = inicio + porPagina;
const datosPagina = estudiantes.slice(inicio, fin);
renderTabla(datosPagina);
```

---

### 🔄 CRUD en Tablas HTML

Las operaciones CRUD que aprendiste en Python tienen equivalentes en JavaScript:

| Operación | Python (CSV) | JavaScript (Array + DOM) |
|------------|--------------|--------------------------|
| **Crear** | `csv.DictWriter()` + `writerow()` | `array.push()` + `renderTabla()` |
| **Leer** | `csv.DictReader()` | `array.filter()` + `tbody.innerHTML` |
| **Actualizar** | Leer → Modificar → `DictWriter` | `array.findIndex()` + `array[index] = nuevo` |
| **Eliminar** | Leer → Filtrar → `DictWriter` | `array.splice()` + `renderTabla()` |

---

### 💾 Persistencia con localStorage

**Python:** Guardas en archivo CSV
```python
import csv

with open('estudiantes.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['nombre', 'nota'])
    writer.writeheader()
    writer.writerows(estudiantes)
```

**JavaScript:** Guardas en localStorage (navegador)
```javascript
// Guardar
localStorage.setItem('estudiantes', JSON.stringify(estudiantes));

// Leer
const guardados = JSON.parse(localStorage.getItem('estudiantes'));
```

---

### ⚡ DOM Manipulation

**Concepto clave:** En JavaScript, modificas el HTML dinámicamente:

```javascript
// Actualizar el cuerpo de la tabla
const tbody = document.getElementById('tablaBody');

// Crear filas HTML desde los datos
tbody.innerHTML = estudiantes.map(est => `
  <tr>
    <td>${est.nombre}</td>
    <td>${est.email}</td>
    <td>${est.nota}</td>
  </tr>
`).join('');
```

**Equivalente Python (para entender):**
```python
# Python no tiene "DOM", pero esto es similar:
html_filas = ""
for est in estudiantes:
    html_filas += f"<tr><td>{est['nombre']}</td><td>{est['email']}</td></tr>"
print(html_filas)
```

---

### 🎯 Plantilla Web S18

La plantilla web de S18 incluye:

1. **Tabla interactiva:** Muestra estudiantes con búsqueda y filtros
2. **Filtrado en tiempo real:** Busca mientras escribes
3. **Ordenamiento:** Click en encabezados para ordenar columnas
4. **Paginación:** Navega entre páginas de datos
5. **Estadísticas:** Total, promedio, máximo, mínimo
6. **localStorage:** Persistencia de datos en el navegador

**Ruta:** `estudiantes/templates/web_sessions/s18/`

**Archivos:**
- `index.html` - Estructura de la página con tabla
- `app.js` - Lógica de filtrado, ordenamiento y paginación
- `styles.css` - Estilos para tabla y filtros

---

### 🔗 Diferencias Clave

| Concepto | Python | JavaScript (Web) |
|-----------|--------|------------------|
| Datos | Archivos CSV/Excel | JSON en memoria/localStorage |
| Mostrar | `print()` o pandas | HTML `<table>` con DOM |
| Interactividad | Re-ejecutar código | Eventos (click, input) |
| Persistencia | Archivos en disco | localStorage |
| Filtrado | Una vez (estático) | Tiempo real (dinámico) |

---

### 🚀 Siguientes Pasos

1. Abre la carpeta `estudiantes/templates/web_sessions/s18/`
2. Abre `index.html` en tu navegador
3. Practica con la tabla de estudiantes:
   - Escribe en el buscador para filtrar
   - Click en encabezados para ordenar
   - Usa los botones de paginación
   - Observa las estadísticas actualizarse
4. Compara el código Python con el código JavaScript en los comentarios

**¡Verás cómo los conceptos de datos que aprendiste en Python se aplican en la web!**

## 🎯 Resumen de la Sesión

En esta práctica intensiva:
1. Construiste un **sistema CRUD completo**
2. Aprendiste a **generar reportes** desde CSV
3. Implementaste **backups** y **logs**
4. Manejaste **errores** de archivos
5. Creaste un **menú interactivo**

---

## ➡️ Próxima Sesión
**Sesión 21: Debug y Buenas Prácticas**

¡Aprenderás a encontrar y corregir errores como un profesional!

---
*📋 Después de esta sesión viene la **Evaluación 4** (Nota 4)*

---
*¡Completaste el Bloque 4: Archivos! 🎉*